##### Copyright 2023 Google LLC

In [5]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [6]:
!pip install -U -q "google-generativeai>=0.8.2"

In [7]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'learnlm-1.5-pro-experimental' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6ItCh0L/QsNGB0LjQsdC+LCDRh9GC0L4g0L7RgtCy0LXRgtC40LvQuFxcblxcbtCg0LDQtNGLINGB0LvRi9GI0LDRgtGMLCDRh9GC0L4g0JLQsNC8INCy0YHRkSDQv9C+0L3RgNCw0LLQuNC70L7RgdGMKVxcblxcbtCf0L7QvdC40LzQsNC10Lwg0Lgg0JLQsNGI0Lgg0L/QtdGA0LXQttC40LLQsNC90LjRjyDQuCDQv9C10YDQtdC20LjQstCw0L3QuNGPINCc0LDRgNC60LAsINCx0LXQt9GD0YHQu9C+0LLQvdC+INGN0YLQviDQstC+0LvQvdC40YLQtdC70YzQvdGL0Lkg0L3QvtCy0YvQuSDRjdGC0LDQvywg0L3QviDQstGB0LUg0LDQtNCw0L/RgtC40YDRg9GO0YLRgdGPINC4INCy0YHRkSDQsdGD0LTQtdGCINGF0L7RgNC+0YjQvi5cXG5cXG7QkNC90L3QsCwg0L/QvtC90Y/Qu9CwINC90LDRgdGH0ZHRgiDQsNC90LPQu9C40LnRgdC60L7Qs9C+LCDQsiDRiNC60L7Qu9GDINC90LDQv9C40YjRgylcXG5cXG7QodC/0LDRgdC40LHQviEifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJgYGBqc29uXFxue1xcbiAgXFxcItC/0LXRgNC10L/QuNGB0LrQsF9pZFxcXCI6IFxcXCIxXFxcIixcXG4gIFxcXCLRg9GA0L7QstC10L3RjF/QstC+0LfRgNCw0LbQtdC90LjRj1xcXCI6IFxcXCI0XFxcIixcXG4gIFxcXCLRhNC+0YDQvNCw0LvRjNC90L7RgdGC0Yxf0YLQtdC60YHRgtCwXFxcIjogXFxcItCd0LjQt9C60LDRj1xcXCIsXFxuICBcXFwi0Y3QvNC+0YbQuNC+0L3QsNC70YzQvdCw0Y9f0L7QutGA0LDRgdC60LBcXFwiOiBcXFwi0J/QvtC30LjRgtC40LLQvdCw0Y9cXFwiLFxcbiAgXFxcItCw0LPRgNC10YHRgdC40LLQvdC+0YHRgtGMXFxcIjogMCxcXG4gIFxcXCLRgtC+0L3QsNC70YzQvdC+0YHRgtGMXFxcIjogMixcXG4gIFxcXCLRgdGC0LjQu9GMX9C+0LHRidC10L3QuNGPXFxcIjogXFxcItCd0LXQudGC0YDQsNC70YzQvdGL0LlcXFwiLFxcbiAgXFxcItGC0LXQvNGLXFxcIjogW1xcbiAgICBcXFwi0KPQtNC+0LLQu9C10YLQstC+0YDQtdC90L3QvtGB0YLRjFxcXCIsXFxuICAgIFxcXCLQn9C+0LTQtNC10YDQttC60LBcXFwiLFxcbiAgICBcXFwi0JDQvdCz0LvQuNC50YHQutC40Lkg0Y/Qt9GL0LpcXFwiLFxcblxcdFxcdFxcXCLQqNC60L7Qu9CwXFxcIlxcbiAgXSxcXG5cXHRcXFwi0LLQtdGA0L7Rj9GC0L3QvtGB0YLRjF/RgtC10LzRi1xcXCI6IFtcXG5cXHRcXHR7XFxuXFx0XFx0XFx0XFxcItGC0LXQvNCwXFxcIjogXFxcItCj0LTQvtCy0LvQtdGC0LLQvtGA0LXQvdC90L7RgdGC0YxcXFwiLFxcblxcdFxcdFxcdFxcXCLQstC10YDQvtGP0YLQvdC+0YHRgtGMXFxcIjogMC44XFxuXFx0XFx0fSxcXG5cXHRcXHR7XFxuXFx0XFx0XFx0XFxcItGC0LXQvNCwXFxcIjogXFxcItCf0L7QtNC00LXRgNC20LrQsFxcXCIsXFxuXFx0XFx0XFx0XFxcItCy0LXRgNC+0Y/RgtC90L7RgdGC0YxcXFwiOiAwLjdcXG5cXHRcXHR9LFxcblxcdFxcdHtcXG5cXHRcXHRcXHRcXFwi0YLQtdC80LBcXFwiOiBcXFwi0JDQvdCz0LvQuNC50YHQutC40Lkg0Y/Qt9GL0LpcXFwiLFxcblxcdFxcdFxcdFxcXCLQstC10YDQvtGP0YLQvdC+0YHRgtGMXFxcIjogMC42XFxuXFx0XFx0fSxcXG5cXHRcXHR7XFxuXFx0XFx0XFx0XFxcItGC0LXQvNCwXFxcIjogXFxcItCo0LrQvtC70LBcXFwiLFxcblxcdFxcdFxcdFxcXCLQstC10YDQvtGP0YLQvdC+0YHRgtGMXFxcIjogMC42XFxuXFx0XFx0fVxcbiBdLFxcbiAgXFxcItGB0YLRgNCw0L3QsF/Qv9C+0LvRjNC30L7QstCw0YLQtdC70Y9cXFwiOiBcXFwi0J3QtdC40LfQstC10YHRgtC90L5cXFwiLFxcbiAgXFxcItGD0YHRgtGA0L7QudGB0YLQstC+X9C/0L7Qu9GM0LfQvtCy0LDRgtC10LvRj1xcXCI6IFxcXCLQndC10LjQt9Cy0LXRgdGC0L3QvlxcXCJcXG59XFxuYGBgIFxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "\u0421\u043f\u0430\u0441\u0438\u0431\u043e, \u0447\u0442\u043e \u043e\u0442\u0432\u0435\u0442\u0438\u043b\u0438\\n\\n\u0420\u0430\u0434\u044b \u0441\u043b\u044b\u0448\u0430\u0442\u044c, \u0447\u0442\u043e \u0412\u0430\u043c \u0432\u0441\u0451 \u043f\u043e\u043d\u0440\u0430\u0432\u0438\u043b\u043e\u0441\u044c)\\n\\n\u041f\u043e\u043d\u0438\u043c\u0430\u0435\u043c \u0438 \u0412\u0430\u0448\u0438 \u043f\u0435\u0440\u0435\u0436\u0438\u0432\u0430\u043d\u0438\u044f \u0438 \u043f\u0435\u0440\u0435\u0436\u0438\u0432\u0430\u043d\u0438\u044f \u041c\u0430\u0440\u043a\u0430, \u0431\u0435\u0437\u0443\u0441\u043b\u043e\u0432\u043d\u043e \u044d\u0442\u043e \u0432\u043e\u043b\u043d\u0438\u0442\u0435\u043b\u044c\u043d\u044b\u0439 \u043d\u043e\u0432\u044b\u0439 \u044d\u0442\u0430\u043f, \u043d\u043e \u0432\u0441\u0435 \u0430\u0434\u0430\u043f\u0442\u0438\u0440\u0443\u044e\u0442\u0441\u044f \u0438 \u0432\u0441\u0451

## Call `generate_content`

In [8]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

InvalidArgument: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [10]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Спасибо, что ответили\n\nРады слышать, что Вам всё понравилось)\n\nПонимаем и Ваши переживания и переживания Марка, безусловно это волнительный новый этап, но все адаптируются и всё будет хорошо.\n\nАнна, поняла насчёт английского, в школу напишу)\n\nСпасибо! 

-------------------------------------------------------------------------------- 

Role: model 

```json\n{\n  \"переписка_id\": \"1\",\n  \"уровень_возражения\": \"4\",\n  \"формальность_текста\": \"Низкая\",\n  \"эмоциональная_окраска\": \"Позитивная\",\n  \"агрессивность\": 0,\n  \"тональность\": 2,\n  \"стиль_общения\": \"Нейтральный\",\n  \"темы\": [\n    \"Удовлетворенность\",\n    \"Поддержка\",\n    \"Английский язык\",\n\t\t\"Школа\"\n  ],\n\t\"вероятность_темы\": [\n\t\t{\n\t\t\t\"тема\": \"Удовлетворенность\",\n\t\t\t\"вероятность\": 0.8\n\t\t},\n\t\t{\n\t\t\t\"тема\": \"Поддержка\",\n\t\t\t\"вероятность\": 0.7\n\t\t},\n\t\t{\n\t\t\t\"тема\": \"Английский язык\",\n\t\t\t\"вероятность\": 0.6\n\t\t},\n\t\t{